<a href="https://colab.research.google.com/github/xorb01/manufacturing-part-classifier/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install ultralytics

In [ ]:
%%writefile /content/drive/MyDrive/yolo_feature/data.yaml
train: /content/drive/MyDrive/yolo_feature/train/images
val: /content/drive/MyDrive/yolo_feature/val/images
test: /content/drive/MyDrive/yolo_feature/test/images

# 클래스 개수
nc: 2
names: ['fsme', 'fsue']

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="/content/drive/MyDrive/yolo_feature/data.yaml",
    epochs=70,
    imgsz=640,
    fliplr=0.5,        # 좌우 반전
  # box=15.0,
    hsv_v=0.4,           # 밝기 변화
    degrees=	90.0,        # 회전 플마 10도
    scale	= 0.5,           # 확대/축소
    shear=2.0,           # 기울기

)

In [ ]:
import os
import shutil
source_path = '/content/runs/detect/train'
destination_path = '/content/drive/MyDrive/yolo_feature_result0108'
if os.path.exists(source_path):
    shutil.copytree(source_path, destination_path)
    print(f"학습 결과 저장")
else:
    print("폴더 탐색 불가.")

In [ ]:
results = model.predict(source='/content/drive/MyDrive/yolo_feature/test/images', save=True)

In [ ]:
results = model.predict(source="/content/runs/detect/predict/FSUE37.jpg", visualize=True)

In [ ]:
!pip install grad-cam

In [ ]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# 모델 설정
model = YOLO('/content/runs/detect/train/weights/best.pt')
model.model.train()

# YOLO 출력을 스칼라로 변환하는 클래스
class YoloTarget(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        results = self.model(x)
        return results[0][:, 4:].max(dim=1)[0]


img_path = '/content/runs/detect/predict/FSME29.jpg' # 이미지 경로 확인

img = cv2.imread(img_path)
img = cv2.resize(img, (640, 640))
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0

input_tensor = torch.from_numpy(np.transpose(rgb_img, (2, 0, 1))).float().unsqueeze(0)
input_tensor.requires_grad = True

# 모델 및 타겟 레이어 설정
wrapped_model = YoloTarget(model.model)
target_layers = [model.model.model[9]]  # Backbone의 마지막 층

# Grad-CAM 실행
cam = GradCAM(model=wrapped_model, target_layers=target_layers)
targets = [ClassifierOutputTarget(0)] # 0: FSME, 1: FSUE

grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :]

# 시각화
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

plt.figure(figsize=(10, 10))
plt.imshow(visualization)
plt.title("Grad-CAM Result: Fixed Scalar Error")
plt.axis('off')
plt.show()